In [ ]:
pip install nltk, emoji, tqdm

# Обработка текста

In [ ]:
import pandas as pd
import numpy as np
import re
import html
import emoji
from tqdm import tqdm

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

русский текст

In [ ]:
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("punkt_tab")

stemmer = SnowballStemmer("russian")
stop_words = set(stopwords.words("russian"))

tqdm.pandas() 

# чистка
def clean_text(text):
    if pd.isnull(text):
        return ""
    
    text = html.unescape(text)
    text = re.sub(r"<.*?>", " ", text)
    # text = re.sub(r"</?[^>]+>", " ", text)
    text = re.sub(r"[^a-zA-Zа-яА-Я]", " ", text)

    return text.lower()

# токенизация и стемминг
def tokenize_and_stem(text):
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t not in stop_words and len(t) > 2]

    stems = [stemmer.stem(t) for t in tokens]

    return " ".join(stems)

def prepare_text(df):
    print("Очистка текстов началась...")
    df["text_clean"] = df["text"].progress_apply(lambda x: tokenize_and_stem(clean_text(x)))
    print("Обработка завершена!")
    return df

df = prepare_text(df)

русский и английский тексты

In [ ]:
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("punkt_tab")

stop_words = set(stopwords.words("russian")) | set(stopwords.words("english"))

stem_ru = SnowballStemmer("russian")
stem_en = SnowballStemmer("english")

def stem_word(word):
    if re.fullmatch(r"[a-zA-Z]+", word):
        return stem_en.stem(word)
    elif re.fullmatch(r"[а-яА-ЯёЁ]+", word):
        return stem_ru.stem(word)
    else:
        return word
    

tqdm.pandas() 

# чистка
def clean_text(text):
    if pd.isnull(text):
        return ""
    
    text = html.unescape(text)
    text = re.sub(r"<.*?>", " ", text)
    text = re.sub(r"[^a-zA-Zа-яА-Я]", " ", text)

    return text.lower()

# токенизация и стемминг
def tokenize_and_stem(text):
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t not in stop_words and len(t) > 2]

    stems = [stem_word(t.lower()) for t in tokens]

    return " ".join(stems)

def prepare_text(df):
    print("Очистка текстов началась...")
    df["text_clean"] = df["text"].progress_apply(lambda x: tokenize_and_stem(clean_text(x)))
    print("Обработка завершена!")
    return df

df = prepare_text(df)

# TF-IDF

## Кластеризация

TF-IDF-vector + SVD + K-means

In [ ]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import re
import html
import emoji

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

In [ ]:
df = pd.read_excel('TextsUnlabeledVar1.xlsx', header=None, names=['text'])

In [ ]:
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("punkt_tab")

stemmer = SnowballStemmer("russian")
stop_words = set(stopwords.words("russian")) | set(stopwords.words("english"))

tqdm.pandas() 

# чистка
def clean_text(text):
    if pd.isnull(text):
        return ""
    
    text = html.unescape(text)
    text = re.sub(r"<.*?>", " ", text)
    text = re.sub(r"[^a-zA-Zа-яА-Я]", " ", text)

    return text.lower()

# токенизация и стемминг
def tokenize_and_stem(text):
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t not in stop_words and len(t) > 2]

    stems = [stemmer.stem(t) for t in tokens]

    return " ".join(stems)

def prepare_text(df):
    print("Очистка текстов началась..")
    df["text_clean"] = df["text"].progress_apply(lambda x: tokenize_and_stem(clean_text(x)))
    print("Обработка завершена!")
    return df

df = prepare_text(df)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans

In [ ]:
vectorizer = TfidfVectorizer(max_df=0.9, min_df=5, ngram_range=(1,2), max_features=100000)
X = vectorizer.fit_transform(df["text_clean"])

svd = TruncatedSVD(n_components=100, random_state=42)
X_svd = svd.fit_transform(X) 

kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
labels = kmeans.fit_predict(X_svd)

df['cluster'] = labels

## Классификация

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import make_scorer, f1_score

In [ ]:
X_texts = df['text_clean']
y = df['label']

In [ ]:
vectorizer = TfidfVectorizer(
    max_df=0.9,
    min_df=5,
    ngram_range=(1,2),
    max_features=10000,
)

X = vectorizer.fit_transform(X_texts)

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scorer = make_scorer(f1_score)

models = [
    ("LogisticRegression", LogisticRegression(
        penalty='l2',
        C=1.0,
        solver='liblinear',
        max_iter=1000,
        random_state=42
    )),
    ("RandomForest", RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        max_features='sqrt',
        bootstrap=True,
        random_state=42
    )),
    ("SVM", SVC(
        kernel='linear',
        C=1.0,
        probability=True,
        random_state=42
    ))
]

In [ ]:
for name, model in models:
    scores = cross_val_score(model, X, y, cv=skf, scoring=scorer)
    print(f"Модель: {name}")
    print(f"F1-score по 5 фолдам: {scores}")
    print(f"Средний F1: {scores.mean():.4f}")
    print("-"*40)

In [ ]:
X_texts = df['text_clean']
y = df['label']

vectorizer = TfidfVectorizer(
    max_df=0.9,
    min_df=5,
    ngram_range=(1,2),
    max_features=10000,
)
X = vectorizer.fit_transform(X_texts)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scorer = make_scorer(f1_score)

models = [
    ("LogisticRegression", LogisticRegression(
        penalty='l2',
        C=1.0,
        solver='liblinear',
        max_iter=1000,
        random_state=42
    )),
    ("RandomForest", RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        max_features='sqrt',
        bootstrap=True,
        random_state=42
    )),
    ("SVM", SVC(
        kernel='linear',
        C=1.0,
        probability=True,
        random_state=42
    ))
]

for name, model in models:
    scores = cross_val_score(model, X, y, cv=skf, scoring=scorer)
    print(f"Модель: {name}")
    print(f"F1-score по 5 фолдам: {scores}")
    print(f"Средний F1: {scores.mean():.4f}")
    print("-"*40)

final_model = LogisticRegression(
    penalty='l2',
    C=1.0,
    solver='liblinear',
    max_iter=1000,
    random_state=42
)
final_model.fit(X, y)


## Регрессия

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import make_scorer, mean_squared_error
import numpy as np

In [ ]:
X_texts = df['text_clean']
y = df['target']  # числовая переменная

In [ ]:
vectorizer = TfidfVectorizer(
    max_df=0.9,
    min_df=5,
    ngram_range=(1,2),
    max_features=10000,
    sublinear_tf=True,
    strip_accents='unicode'
)
X = vectorizer.fit_transform(X_texts)

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

models = [
    ("LinearRegression", LinearRegression()),
    ("Ridge", Ridge(alpha=1.0, solver='auto', random_state=42)),
    ("Lasso", Lasso(alpha=0.01, max_iter=1000, random_state=42)),
    ("RandomForest", RandomForestRegressor(
        n_estimators=300,
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        max_features='sqrt',
        bootstrap=True,
        random_state=42
    )),
    ("SVR", SVR(
        kernel='linear',
        C=1.0,
        epsilon=0.1
    ))
]

In [ ]:
for name, model in models:
    scores = cross_val_score(model, X, y, cv=kf, scoring=mse_scorer)
    print(f"Модель: {name}")
    print(f"MSE по 5 фолдам: {-scores}")  # умножаем на -1, потому что greater_is_better=False
    print(f"Средний MSE: {-scores.mean():.4f}")
    print("-"*40)

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import make_scorer, mean_squared_error
import numpy as np

X_texts = df['text_clean']
y = df['target']  # числовая переменная

vectorizer = TfidfVectorizer(
    max_df=0.9,
    min_df=5,
    ngram_range=(1,2),
    max_features=10000,
    sublinear_tf=True,
    strip_accents='unicode'
)
X = vectorizer.fit_transform(X_texts)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

models = [
    ("LinearRegression", LinearRegression()),
    ("Ridge", Ridge(alpha=1.0, solver='auto', random_state=42)),
    ("Lasso", Lasso(alpha=0.01, max_iter=1000, random_state=42)),
    ("RandomForest", RandomForestRegressor(
        n_estimators=300,
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        max_features='sqrt',
        bootstrap=True,
        random_state=42
    )),
    ("SVR", SVR(
        kernel='linear',
        C=1.0,
        epsilon=0.1
    ))
]

for name, model in models:
    scores = cross_val_score(model, X, y, cv=kf, scoring=mse_scorer)
    print(f"Модель: {name}")
    print(f"MSE по 5 фолдам: {-scores}")  # умножаем на -1, потому что greater_is_better=False
    print(f"Средний MSE: {-scores.mean():.4f}")
    print("-"*40)

## Ключевые слова

In [ ]:
import numpy as np

# X_tfidf — твоя TF-IDF матрица (разреженная)
# vectorizer — твой TfidfVectorizer

# 1. суммируем TF-IDF по всем документам
tfidf_sum = X.sum(axis=0)  # shape = (1, num_features)

# 2. имена признаков (слов)
feature_names = vectorizer.get_feature_names_out()

# 3. сортируем по убыванию
top_idx = np.array(tfidf_sum).ravel().argsort()[::-1]

# 4. выводим топ N слов
N = 50
top_words = [(feature_names[i], tfidf_sum[0, i]) for i in top_idx[:N]]

print("Топ-50 слов по всему корпусу:")
for word, score in top_words:
    print(word, score)

In [ ]:
n_unique_words = len(vectorizer.vocabulary_)
print(n_unique_words)

In [ ]:
feature_names = vectorizer.get_feature_names_out()
print(len(feature_names))
print(feature_names[:20])

# Всё остальное

## На случай невозможного

In [ ]:
# перевод смайликов в текст
def replace_emoji(text):
    return emoji.demojize(text, language="ru")


# тональность
POS_EMOJI = " позитив "
NEG_EMOJI = " негатив "

def replace_emoji_with_sentiment(text):
    for ch in text:
        if ch in emoji.EMOJI_DATA:
            if ch in {"😀", "😃", "😊", "😍", "😁", "😂", "🤣", "❤️", "👍"}:
                text = text.replace(ch, POS_EMOJI)
            elif ch in {"😡", "😠", "😢", "😭", "👎", "💔"}:
                text = text.replace(ch, NEG_EMOJI)
            else:
                text = text.replace(ch, " EMOJI_NEUTRAL ")
    return text

# чистка
def clean_text(text):
    if pd.isnull(text):
        return ""
    
    text = html.unescape(text)
    text = re.sub(r"<.*?>", " ", text)
    #text = replace_emoji_with_sentiment(text)
    text = re.sub(r"[^a-zA-Zа-яА-Я0-9]", " ", text)

    return text.lower()

# удаление чисел
text = re.sub(r"[^a-zA-Zа-яА-Я]", " ", text)

## Мусор

### кластеризация

In [ ]:
feature_names = vectorizer.get_feature_names_out()
print(len(feature_names))
print(feature_names[:20])

In [ ]:
doc_vector = X[0]
feature_names = vectorizer.get_feature_names_out()
sorted_items = sorted(zip(feature_names, doc_vector.toarray()[0]), key=lambda x: x[1], reverse=True)
print(sorted_items[:50]) 

In [ ]:
import numpy as np

X = vectorizer.transform(df["text_clean"])  # TF-IDF матрица

feature_names = vectorizer.get_feature_names_out()

# средний TF-IDF каждого слова по всем документам
mean_tfidf = np.asarray(X.mean(axis=0)).ravel()

# сортировка
sorted_features = sorted(
    zip(feature_names, mean_tfidf),
    key=lambda x: x[1],
    reverse=True
)

# топ-20 самых "важных" слов корпуса
print(sorted_features[:20])


In [ ]:
import numpy as np

feature_names = vectorizer.get_feature_names_out()
top_words = {}

for cluster_id in np.unique(labels):
    idxs = np.where(labels == cluster_id)[0]
    tfidf_sum = X[idxs].sum(axis=0)

    # топ-10 слов
    top_idx = np.array(tfidf_sum).ravel().argsort()[::-1][:10]
    top_words[cluster_id] = [feature_names[i] for i in top_idx]

print("Топ-10 слов по кластерам:")
for cid, words in top_words.items():
    print(f"Кластер {cid}: {words}")



cluster_reviews = None
for cid, words in top_words.items():
    if any(w in words for w in ['отел','номер','завтрак','персона']):
        cluster_reviews = cid
        break


num_reviews = (df['cluster'] == cluster_reviews).sum()
print(f"Оценка количества отзывов: {num_reviews}")


### обработка текста

In [ ]:
'''
stop_words = set(stopwords.words("russian")) | set(stopwords.words("english"))

# нормализация названий
def preprocess_app_name(name):
    if pd.isnull(name):
        return ""
    return re.sub(r"[^a-zA-Zа-яА-Я0-9]", " ", name).lower().strip()

# что в итоге получается
def prepare_text(df):
    df["app_name_clean"] = df["app_name"].apply(preprocess_app_name)
    df["desc_clean"] = df["shortDescription"].fillna("") + " " + df["full_description"].fillna("")
    df["desc_clean"] = df["desc_clean"].apply(lambda x: tokenize_and_stem(clean_text(x)))
    df["text_clean"] = df["app_name_clean"] + " " + df["desc_clean"]
    return df
'''

'''
nltk==3.8.1
nltk.download("punkt")
'''

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Пример текстов
texts = [
    "отель хороший номер",
    "номер был чистый",
    "отель чистый и уютный"
]

# Инициализация CountVectorizer
vectorizer = CountVectorizer(
    max_df=0.9,        # игнорировать слишком частые слова
    min_df=1,          # игнорировать слова, встречающиеся меньше раз
    ngram_range=(1,1)  # 1-граммы (отдельные слова)
)

# Преобразуем тексты в матрицу частот
X = vectorizer.fit_transform(texts)

# Словарь слов (фичи)
feature_names = vectorizer.get_feature_names_out()
print("Словарь слов:")
print(feature_names)

# Вектор документа
print("\nМатрица признаков (CountVectorizer):")
print(X.toarray())

# Каждая строка = документ, каждая колонка = слово, число = количество раз, которое слово встретилось
